# Semantic Segmentation Quiz

### Maintainer : Wonkwang Lee (wonkwang.lee@kaist.ac.kr)

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=(1, 224, 224, 3))
label = tf.broadcast_to(tf.expand_dims(tf.expand_dims(tf.one_hot(1, 21), -1), -1), (1, 21, 224, 224))

# Quiz 1: Building toy FCN blocks

![FCN](http://deeplearning.net/tutorial/_images/cat_segmentation.png)


## Instruction:
Build toy FCN blocks using `models.Sequential()`, `layers.Conv2D`, `layers.Maxpooling2D`, `layer.UpSampling2D`. <br>
Please refer to the printed output for `fcn.summary()`

#### hint1 : https://www.tensorflow.org/beta/tutorials/images/intro_to_cnns
#### hint2 : https://keras.io/layers/convolutional/#conv2d
#### hint3 : https://keras.io/layers/pooling/#maxpooling2d
#### hint4 : https://keras.io/layers/convolutional/#upsampling2d

In [52]:
def FCN():
    model = models.Sequential()
    model.add(layers.Conv2D(96, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    #model.add(...)
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(4096, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(4096, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(21, (3,3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((8, 8)))
    return model




In [53]:
fcn = FCN()
fcn.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_86 (Conv2D)           (None, 224, 224, 96)      2688      
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 112, 112, 96)      0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 112, 112, 256)     221440    
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 56, 56, 256)       0         
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 56, 56, 384)       885120    
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 56, 56, 384)       1327488   
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 56, 56, 256)     

In [55]:
print(x.shape)

(1, 224, 224, 3)


In [56]:
prediction = fcn(x)
prediction

<tf.Tensor 'sequential_25/up_sampling2d_1/ResizeNearestNeighbor:0' shape=(1, 224, 224, 21) dtype=float32>

---

# Quiz 2: Preprocess data to calculate cross-entropy loss


#### hint1 : https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy
#### hint2 : https://www.tensorflow.org/api_docs/python/tf/transpose

In [73]:
def preprocess_label(label):
    ''' write down proper pre-processing operation here'''
    #label = ...
    label = tf.transpose(label, perm=(0, 2, 3, 1))
    return label

def calculate_loss(prediction, label):
    loss = tf.losses.softmax_cross_entropy(label, prediction)
    return loss

In [74]:
print('prediction shape :', prediction.shape)
print('label shape :', label.shape)

prediction shape : (1, 224, 224, 21)
label shape : (1, 21, 224, 224)


In [75]:
label = preprocess_label(label)
print('label shape :', label.shape)

label shape : (1, 224, 224, 21)


In [76]:
loss = calculate_loss(prediction, label)
loss

W0801 16:04:21.529071 140231406806848 deprecation.py:323] From /home/cmo05/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'softmax_cross_entropy_loss_2/value:0' shape=() dtype=float32>

---

# Quiz 3: Building toy DeconvNet

![deconvnet](https://miro.medium.com/max/1400/1*LW8Anre45o9nfamxIVTY8Q.png)

## Instruction:
Build toy DeconvNet blocks using `models.Sequential()`, `layers.Conv2D`, `layers.Maxpooling2D`, `layer.Conv2DTranspose`. <br>
Please refer to the printed output for `deconvnet.summary()` <br>
<br>
Note: we use `layer.Conv2DTranspose` for upsampling feature maps instead of un-pooling operation

In [117]:
def DeconvNet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    #model.add(...)
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2DTranspose(512, (3,3), strides=2, activation='relu', padding='same'))
    model.add(layers.Conv2DTranspose(256, (3,3), strides=2, activation='relu', padding='same'))
    model.add(layers.Conv2DTranspose(128, (3,3), strides=2, activation='relu', padding='same'))
    model.add(layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu', padding='same'))   
    model.add(layers.Conv2DTranspose(21, (3,3), strides=2, activation='relu', padding='same'))    
    return model

In [118]:
deconvnet = DeconvNet()
deconvnet.summary()

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_182 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_126 (MaxPoolin (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_183 (Conv2D)          (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_127 (MaxPoolin (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_184 (Conv2D)          (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_128 (MaxPoolin (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_185 (Conv2D)          (None, 28, 28, 256)     

In [119]:
prediction = deconvnet(x)
prediction

<tf.Tensor 'sequential_44/conv2d_transpose_15/Relu:0' shape=(1, 224, 224, 21) dtype=float32>

---

# Quiz 4 : Feature Pyramid

<img src="toy_fpn.png" width="600">

## Instruction
Implement `upsample_and_merge` function

#### hint1 : https://www.tensorflow.org/api_docs/python/tf/concat

In [123]:
conv_block1 = models.Sequential()
conv_block1.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
conv_block1.add(layers.MaxPool2D((2,2)))

conv_block2 = models.Sequential()
conv_block2.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
conv_block2.add(layers.MaxPool2D((2,2)))

In [124]:
out1 = conv_block1(x)
out1

<tf.Tensor 'sequential_47/max_pooling2d_133/MaxPool:0' shape=(1, 112, 112, 32) dtype=float32>

In [125]:
out2 = conv_block2(out1)
out2

<tf.Tensor 'sequential_48/max_pooling2d_134/MaxPool:0' shape=(1, 56, 56, 64) dtype=float32>

In [138]:
def upsample_and_merge(small, large):
    '''write down codes here'''
    # merged = ...
    small = tf.keras.layers.UpSampling2D((2,2), interpolation='nearest')(small)
    merged = tf.concat([small, large], 3)
    return merged

In [139]:
upsample_and_merge(out2, out1)

<tf.Tensor 'concat_2:0' shape=(1, 112, 112, 96) dtype=float32>

---

# Quiz 5 : Building blocks with atrous convolution

![fcn_atrous](fcn_atrous.png)
#### hint1 : https://keras.io/layers/convolutional/#conv2d

In [152]:
def Conv_Atrous():
    model = models.Sequential()
    ''' implement remaining modules here. note that the last four modules should be atrous convs'''
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    #model.add(...)
    #...
    # add atrous convolution with rate 2
    #model.add(...)
    # add atrous convolution with rate 4
    #model.add(...)
    # add atrous convolution with rate 8
    #model.add(...)
    # add atrous convolution with rate 16
    #model.add(...)
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))    
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))      
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))   
    model.add(layers.Conv2D(512, (3, 3), dilation_rate=(2), activation='relu', padding='same'))  
    model.add(layers.Conv2D(512, (3, 3), dilation_rate=(4), activation='relu', padding='same'))     
    model.add(layers.Conv2D(512, (3, 3), dilation_rate=(8), activation='relu', padding='same'))      
    model.add(layers.Conv2D(512, (3, 3), dilation_rate=(16), activation='relu', padding='same'))        
    return model

In [153]:
conv_atrous = Conv_Atrous()
conv_atrous.summary()

Model: "sequential_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_228 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_166 (MaxPoolin (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_229 (Conv2D)          (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_167 (MaxPoolin (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_230 (Conv2D)          (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_168 (MaxPoolin (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_231 (Conv2D)          (None, 28, 28, 256)     

---

# Quiz 6 : Building toy DeepLab

![aspp](aspp.png)

## Instruction
Complete ``DeepLab_Front`, conv_with_dilation_rate2`, `conv_with_dilation_rate4`, `conv_with_dilation_rate8`, `conv_with_dilation_rate16`, and `aspp`, where `aspp` is simply computed by concatenating all the `feature_map_rateX`

In [154]:
def DeepLab_Front():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    #model.add(...)
    model.add(layers.MaxPooling2D((2,2)))    
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same')) 
    model.add(layers.MaxPooling2D((2,2)))    
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))      
    model.add(layers.MaxPooling2D((2,2)))    
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))      
    model.add(layers.MaxPooling2D((2,2)))    
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))       
    return model

In [155]:
front = DeepLab_Front()
front.summary()

Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_237 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_171 (MaxPoolin (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_238 (Conv2D)          (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_172 (MaxPoolin (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_239 (Conv2D)          (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_173 (MaxPoolin (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_240 (Conv2D)          (None, 28, 28, 256)     

In [161]:
''' write down codes here '''
#conv_with_dilation_rate2 = ...
#conv_with_dilation_rate4 = ...
#conv_with_dilation_rate8 = ...
#conv_with_dilation_rate16 = ...
conv_with_dilation_rate2 = layers.Conv2D(512, (3, 3), dilation_rate=(2), activation='relu', padding='same')
conv_with_dilation_rate4 = layers.Conv2D(512, (3, 3), dilation_rate=(4), activation='relu', padding='same')
conv_with_dilation_rate8 = layers.Conv2D(512, (3, 3), dilation_rate=(8), activation='relu', padding='same')
conv_with_dilation_rate16 = layers.Conv2D(512, (3, 3), dilation_rate=(16), activation='relu', padding='same')

In [162]:
feature_map = front(x)
feature_map_rate2 = conv_with_dilation_rate2(feature_map)
feature_map_rate4 = conv_with_dilation_rate4(feature_map)
feature_map_rate8 = conv_with_dilation_rate8(feature_map)
feature_map_rate16 = conv_with_dilation_rate16(feature_map)

In [163]:
sppp = tf.concat([feature_map_rate2, feature_map_rate4, feature_map_rate8, feature_map_rate16], axis=3)
sppp

<tf.Tensor 'concat_4:0' shape=(1, 14, 14, 2048) dtype=float32>

---

# Quiz 7 : Building toy Unet

![unet](simple_unet.png)

## Instruction
implement a simple U-Net module using `ds`, `upsample`, and `deconv` module

In [164]:
def UNet_layer3(x):
    
    ds = layers.MaxPooling2D((2, 2))
    upsample = layers.UpSampling2D((2, 2))
    deconv = layers.Conv2DTranspose(3, (3, 3), activation='relu', strides=2, padding='same')
    
    out = ds(x)
    out_up = upsample(out)
    out_large = deconv(out)
    out = tf.concat([out_up, out_large], axis=3)
    return out

In [165]:
UNet_layer3(x)

<tf.Tensor 'concat_5:0' shape=(1, 224, 224, 6) dtype=float32>